In [38]:
import rdkit 
from rdkit.Chem import MolFromSmiles as get_mol
from rdkit.Chem import rdDistGeom
import numpy as np 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader, random_split
import multiprocessing

%run utils.py
%run model.py

In [25]:
smi_list = get_smi_list('data/ADAGRASIB_SMILES.txt')
coor_list = parallel_f(get_atom_pos, smi_list)
longest_smi = len(max(smi_list, key = len))
longest_coor = len(max(coor_list, key = len))
n_coor_list = [normalize(c) for c in coor_list]
np_coor_list = [pad(c, longest_coor) for c in n_coor_list]

[22:41:59] UFFTYPER: Unrecognized atom type: Ba (0)


In [29]:
class MyDataset(Dataset) : 
    def __init__(self, smi_list, coor_list) : 
        self.smi_list = smi_list
        self.coor_list = coor_list

    def __len__(self) :
        return len(self.smi_list)
    
    def __getitem__(self, idx) :
        return self.smi_list[idx], self.coor_list[idx]

In [39]:
BATCH_SIZE = 128 
dataset = MyDataset(smi_list, np_coor_list)
train_set, val_set, test_set = random_split(dataset, [0.9, 0.05, 0.05])

train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = True)

In [40]:
for smi, coor in train_loader : 
    print(smi.shape, coor.shape)
    break

AttributeError: 'tuple' object has no attribute 'shape'

In [37]:
rdkit.Chem.MolToSmiles(get_mol(smi_list[0]))

'Cn1cc(B2OC(C)(C)C(C)(C)O2)cn1'